In [ ]:
!pip -q install \
pymongo \
groq \
sentence-transformers \
langchain\
langchain-community \
langchain-google-genai\
google-generativeai \
mlflow \
pyngrok

In [ ]:
import re
import random
import pandas as pd
import logging
from typing import List, Union, Any, Callable, Dict, Tuple, Optional
from dataclasses import dataclass
from pymongo import MongoClient

import time
import mlflow
from pyngrok import ngrok, conf
import subprocess

import google.generativeai as genai
from google.genai.types import HttpOptions, GenerateContentConfig
from google.genai import Client as GeminiClient
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage

# 1/ Configuration

In [ ]:
# Config logger
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Config MongoDB
MONGO_URI = ...
MONGO_DATABASE_NAME  = 'deeplearning_ai_news'

# Goole variables
GOOGLE_API_KEY = [
    'api key 1',
    'api key 2',
    ...
]
GEMINI_MODEL_NAME = 'gemini-2.0-flash-lite'
MAX_ATTEMPTS = 5
DELAY_TIME = 10

# Config MLFlow
MLFLOW_TRACKING_URI = "https://mlflow-render-server.onrender.com"
MLFLOW_EXPERIMENT = "FActScore Evaluation"

In [ ]:
# Access to MongoDB
mongo_client = MongoClient(MONGO_URI)
mongo_db = mongo_client[MONGO_DATABASE_NAME]
mongo_collections = mongo_db.list_collection_names()
print('All collections existing: ', mongo_collections)

# Get all samples
data = []
for col in mongo_collections:
    data.extend(mongo_db[col].find())

print("Number of data samples: ", len(data))

All collections existing:  ['science', 'data-points', 'hardware', 'ml-research', 'business', 'culture']
Number of data samples:  982


In [ ]:
# Get one example
sample = data[0]

# Get content and summary
content = sample['content']
summary = sample['summary']

# Print
print(f"Summary: {summary}")
print('---')
print(f"Content: {content}")

Summary: Dưới đây là bản tóm tắt bài viết dưới dạng bullet points:

*   Các nhà nghiên cứu đã phát triển AI Scientist, một quy trình làm việc sử dụng các mô hình ngôn ngữ lớn (LLM) để tạo ra các ý tưởng nghiên cứu AI, viết mã để thử nghiệm và ghi lại kết quả.
*   AI Scientist sử dụng các LLM như Claude Sonnet 3.5, GPT-4o, DeepSeek Coder và LLama 3.1 405B để tạo ra các bài báo trong ba lĩnh vực: mô hình hóa hình ảnh khuếch tán, mô hình ngôn ngữ dựa trên transformer và "grokking".
*   GPT-4o được sử dụng để đánh giá các bài báo được tạo ra theo các tiêu chuẩn của hội nghị NeurIPS, cho thấy tiềm năng của AI trong việc tạo ra các nghiên cứu mới với kết quả thử nghiệm thực tế.
---
Content: While some observers argue that large language models can’t produce truly original output, new work prompted them to generate novel scientific research.

What’s new:Researchers proposedAI Scientist, an agentic workflow that directs large language models to generate ideas for AI research, produce code to t

In [ ]:
# Regex to extract sentence behind *
def extract_sentence(text: str = None) -> List[str]:
    sentences = []
    if text:
        sentences = re.findall(r"\*\s+(.*)", text)

    return sentences

# Test
extract_sentence(summary)

['Các nhà nghiên cứu đã phát triển AI Scientist, một quy trình làm việc sử dụng các mô hình ngôn ngữ lớn (LLM) để tạo ra các ý tưởng nghiên cứu AI, viết mã để thử nghiệm và ghi lại kết quả.',
 'AI Scientist sử dụng các LLM như Claude Sonnet 3.5, GPT-4o, DeepSeek Coder và LLama 3.1 405B để tạo ra các bài báo trong ba lĩnh vực: mô hình hóa hình ảnh khuếch tán, mô hình ngôn ngữ dựa trên transformer và "grokking".',
 'GPT-4o được sử dụng để đánh giá các bài báo được tạo ra theo các tiêu chuẩn của hội nghị NeurIPS, cho thấy tiềm năng của AI trong việc tạo ra các nghiên cứu mới với kết quả thử nghiệm thực tế.']

# 2/ LLM Communicator

In [ ]:
class GeminiLLM:
    def __init__(self, model_name: str, api_keys: List[str]):
        self.model_name = model_name
        self.api_keys = api_keys
        self.current_key_index = 0
        self.model = self._setup_model(self.api_keys[self.current_key_index])

    def _setup_model(self, api_key: str):
        return ChatGoogleGenerativeAI(
            model=self.model_name,
            google_api_key=api_key,
            temperature=0
        )

    def _rotate_key(self):
        self.current_key_index = (self.current_key_index + 1) % len(self.api_keys)
        logger.warning(f"Rotating API key... Now using key index: {self.current_key_index}")
        self.model = self._setup_model(self.api_keys[self.current_key_index])

    def generate(self, query: str, system_prompt: Optional[str] = None) -> str:
        messages = []
        if system_prompt:
            messages.append(SystemMessage(content=system_prompt))
        messages.append(HumanMessage(content=query))

        max_attempts = len(self.api_keys) + MAX_ATTEMPTS
        attempt = 0

        while attempt < max_attempts:
            try:
                # logger.info(f"Gemini generating answer with key index {self.current_key_index}...")
                return self.model.invoke(messages).content

            except Exception as e:
                error_message = str(e)
                logger.error(f"API key index {self.current_key_index} failed: {error_message}")

                # Nếu lỗi là do vượt quota (429)
                if "429" in error_message and "quota" in error_message.lower():
                    logger.warning("Quota limit hit. Waiting before retry (20s)...")
                    time.sleep(30)

                attempt += 1
                self._rotate_key()

        raise RuntimeError("All API keys failed. Please check your quota or keys.")

# 3/ FActScore Setup

In [ ]:
@dataclass
class FactCheckResult:
    claim: str
    is_correct: bool
    evidence: str
    explanation: str

In [ ]:
class FactScoreEvaluator:
    def __init__(self, gemini: GeminiLLM):
        self.gemini = gemini

        # System prompt
        self.system_prompt = (
            "You are a fact-checking assistant. "
            "You verify if a given factual claim is correct based on provided content. "
            "You respond with a Yes/No answer, explanation, and specific evidence."
        )

    def _build_user_prompt(self, content: str, claim: str) -> str:
        return f"""
Content:
\"\"\"{content.strip()}\"\"\"

Claim: "{claim}"

Instructions:
- Answer with "Yes" if the claim is factually correct based on the content, or "No" if it is not.
- Then explain your reasoning briefly.
- Then quote the sentence(s) from the content that support your decision (if applicable).

Format:
Answer: <Yes/No>
Explanation: <your explanation>
Evidence: <quoted evidence from content>
"""

    def evaluate(self, content: str, claims: List[str]) -> Dict:
        results = []

        for claim in claims:
            user_prompt = self._build_user_prompt(content, claim)
            response = self.gemini.generate(query=user_prompt, system_prompt=self.system_prompt)

            answer = "No"
            explanation = ""
            evidence = ""

            try:
                for line in response.strip().splitlines():
                    if line.startswith("Answer:"):
                        answer = line.split(":", 1)[1].strip()
                    elif line.startswith("Explanation:"):
                        explanation = line.split(":", 1)[1].strip()
                    elif line.startswith("Evidence:"):
                        evidence = line.split(":", 1)[1].strip()
            except Exception as e:
                logger.warning(f"Failed to parse Gemini response: {e}")

            results.append(FactCheckResult(
                claim=claim,
                is_correct=answer.lower() == "yes" or "yes" in answer.lower(),
                explanation=explanation,
                evidence=evidence
            ))

        correct_count = sum(1 for r in results if r.is_correct)
        total = len(claims)
        factscore = round(correct_count / total, 4) if total > 0 else 0.0

        return {
            "factscore": factscore,
            "details": results
        }

In [ ]:
# Initialize Gemini LLM
gemini_llm = GeminiLLM(
    model_name=GEMINI_MODEL_NAME,
    api_keys=GOOGLE_API_KEY
)

# Test factscore
evaluator = FactScoreEvaluator(gemini_llm)
results = evaluator.evaluate(content=content, claims=extract_sentence(summary))
print(results)

{'factscore': 1.0, 'details': [FactCheckResult(claim='Các nhà nghiên cứu đã phát triển AI Scientist, một quy trình làm việc sử dụng các mô hình ngôn ngữ lớn (LLM) để tạo ra các ý tưởng nghiên cứu AI, viết mã để thử nghiệm và ghi lại kết quả.', is_correct=True, evidence='"What’s new:Researchers proposedAI Scientist, an agentic workflow that directs large language models to generate ideas for AI research, produce code to test them, and document the enquiry."', explanation='The claim accurately describes the functionality of AI Scientist as described in the text.'), FactCheckResult(claim='AI Scientist sử dụng các LLM như Claude Sonnet 3.5, GPT-4o, DeepSeek Coder và LLama 3.1 405B để tạo ra các bài báo trong ba lĩnh vực: mô hình hóa hình ảnh khuếch tán, mô hình ngôn ngữ dựa trên transformer và "grokking".', is_correct=True, evidence='"How it works: The authors used Claude Sonnet 3.5, GPT-4o, DeepSeek Coder, and LLama 3.1 405B to generate papers in three categories: diffusion image modeling

In [ ]:
print("FactScore:", results["factscore"])
for detail in results["details"]:
    print(f"- Claim: {detail.claim}")
    print(f"\tCorrect: {detail.is_correct}")
    print(f"\tExplanation: {detail.explanation}")
    print(f"\tEvidence: {detail.evidence}")
    print()

FactScore: 1.0
- Claim: Các nhà nghiên cứu đã phát triển AI Scientist, một quy trình làm việc sử dụng các mô hình ngôn ngữ lớn (LLM) để tạo ra các ý tưởng nghiên cứu AI, viết mã để thử nghiệm và ghi lại kết quả.
	Correct: True
	Explanation: The claim accurately describes the functionality of AI Scientist as described in the text.
	Evidence: "What’s new:Researchers proposedAI Scientist, an agentic workflow that directs large language models to generate ideas for AI research, produce code to test them, and document the enquiry."

- Claim: AI Scientist sử dụng các LLM như Claude Sonnet 3.5, GPT-4o, DeepSeek Coder và LLama 3.1 405B để tạo ra các bài báo trong ba lĩnh vực: mô hình hóa hình ảnh khuếch tán, mô hình ngôn ngữ dựa trên transformer và "grokking".
	Correct: True
	Explanation: The content explicitly states that AI Scientist used the listed LLMs to generate papers in the specified three categories.
	Evidence: "How it works: The authors used Claude Sonnet 3.5, GPT-4o, DeepSeek Coder,

# 4/ Run with MLflow

In [ ]:
# Connect to MongoDB
mongo_client = MongoClient(MONGO_URI)
mongo_db = mongo_client[MONGO_DATABASE_NAME]
mongo_collections = mongo_db.list_collection_names()
print('All collections existing: ', mongo_collections)

All collections existing:  ['science', 'data-points', 'hardware', 'ml-research', 'business', 'culture']


In [ ]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(MLFLOW_EXPERIMENT)

for collection in mongo_collections:
    logger.info(f"Processing collection: {collection}...")

    # Try get all data samples from collection
    all_samples = []
    try:
        all_samples.extend(mongo_db[collection].find())
        logger.info(f"Got all samples ({len(all_samples)} samples) from collection: {collection}")
    except Exception as e:
        raise e

    # Initialize result storage
    all_rows = []
    fact_scores = []

    with mlflow.start_run(run_name=collection):
        # Log model information
        mlflow.log_param("llm_model", gemini_llm.model_name)
        mlflow.log_param("temperature", 0)

        for idx, data in enumerate(all_samples):
            data_id = str(data.get('_id', f"sample_{idx}"))
            content = data.get('content', None)
            summary = data.get('summary', None)

            if not content or not summary:
                logger.warning(f"Invalid data sample: {data_id}")
                continue

            # Evaluate with tries
            results = None
            for attempt in range(1, MAX_ATTEMPTS + 1):
                try:
                    claims = extract_sentence(summary)
                    # Check claims
                    if not claims or not isinstance(claims, List):
                        logger.warning(f"No claims extracted for sample: {data_id}")
                        continue
                    results = evaluator.evaluate(content=content, claims=claims)
                    break
                except Exception as e:
                    logger.warning(f"[Attempt {attempt}/{MAX_ATTEMPTS}] Error evaluating sample {data_id}: {e}")

                    if attempt < MAX_ATTEMPTS:
                        logger.info(f"Retrying after {DELAY_TIME} seconds...")
                        time.sleep(DELAY_TIME)
                    else:
                        logger.error(f"Failed after {MAX_ATTEMPTS} attempts for sample {data_id}.")

            # Check results
            if results is None:
                logger.warning(f"Skipping sample {data_id} due to failure.")
                continue
            fact_scores.append(results["factscore"])

            for fact in results["details"]:
                all_rows.append({
                    "data_id": data_id,
                    "claim": fact.claim,
                    "is_correct": fact.is_correct,
                    "explanation": fact.explanation,
                    "evidence": fact.evidence,
                    "score": 1.0 if fact.is_correct else 0.0
                })

        # Logging metrics
        avg_score = round(sum(fact_scores) / len(fact_scores), 4) if fact_scores else 0.0
        mlflow.log_metric("avg_factscore", avg_score)
        mlflow.log_metric("num_samples", len(fact_scores))
        mlflow.log_metric("num_claims", len([r["claim"] for r in all_rows]))
        mlflow.log_metric("num_correct", sum([r["score"] for r in all_rows]))

        # Logging artifacts
        if all_rows:
            df = pd.DataFrame(all_rows)
            artifact_path = f"details_{collection}.csv"
            df.to_csv(artifact_path, index=False)
            mlflow.log_artifact(artifact_path)
        else:
            logger.warning(f"No valid rows to log for collection {collection}!!!")

        logger.info(f"Completed logging for collection: {collection} <3")

WARNI [langchain_google_genai.chat_models] Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 30
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 19
}
].
ERROR [__main__] API key index 0 failed: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error,

🏃 View run ml-research at: https://mlflow-render-server.onrender.com/#/experiments/3/runs/2c50d77556bc41cb8b44617db0b09dc5
🧪 View experiment at: https://mlflow-render-server.onrender.com/#/experiments/3


KeyboardInterrupt: 